In [ ]:
from ultralytics import YOLO
import cv2
import math

# PC Webcam'ini başlat
pc_kamera = cv2.VideoCapture(0)
pc_kamera.set(3, 640)
pc_kamera.set(4, 480)

# Telefon kamerasını başlat
telefon_kamera_adresi = "http://172.16.3.109:8080/video"  # Telefonunuzun IP Webcam uygulamasının sağladığı adresi kullanın
telefon_kamera = cv2.VideoCapture(telefon_kamera_adresi)

# YOLO modelini başlat
model = YOLO("yolo-Weights/yolov8n.pt")

# Nesne sınıfları
sinif_isimleri = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]  # Sınıf isimleriniz

while True:
    # PC Webcam'inden görüntü al
    pc_basari, pc_resim = pc_kamera.read()
    pc_sonuclar = model(pc_resim, stream=True)

    for r in pc_sonuclar:
        pc_kutular = r.boxes

        for pc_kutu in pc_kutular:
            x1, y1, x2, y2 = pc_kutu.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Algılanan nesnenin sınırlayıcı kutusunu çiz
            cv2.rectangle(pc_resim, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Algılama güvenini hesapla
            guven = math.ceil((pc_kutu.conf[0] * 100)) / 100
            print("Güven --->", guven)

            # Algılanan sınıfın adını al
            sinif = int(pc_kutu.cls[0])
            print("Sınıf adı -->", sinif_isimleri[sinif])

            # Algılanan nesnenin sınıf adını ve güvenini ekrana yazdır
            metin = f"{sinif_isimleri[sinif]}: {guven:.2f}"
            org = (x1, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontOlcek = 0.5
            renk = (255, 0, 0)
            kalinlik = 1

            cv2.putText(pc_resim, metin, org, font, fontOlcek, renk, kalinlik)

    # Telefon kamerasından görüntü al
    telefon_basari, telefon_resim = telefon_kamera.read()

    # Telefon kamerasından başarıyla görüntü alındı mı kontrol et
    if telefon_basari:
        # Görüntü boyutlarını eşitle
        telefon_resim = cv2.resize(telefon_resim, (pc_resim.shape[1], pc_resim.shape[0]))

        # Görüntüleri yatay olarak birleştir
        birlesik_resim = cv2.hconcat([pc_resim, telefon_resim])

        # Görüntüyü göster
        cv2.imshow('Bilgisayar Kamerası', birlesik_resim)
    else:
        print("Telefon kamerasından görüntü alınamadı.")

    # 'q' tuşuna basıldığında döngüden çık
    if cv2.waitKey(1) == ord('q'):
        break

# Kameraları serbest bırak ve OpenCV pencerelerini kapat
pc_kamera.release()
telefon_kamera.release()
cv2.destroyAllWindows()

DETECTION WITH PC AND WEBCAM


In [ ]:
from datetime import datetime
from ultralytics import YOLO
import cv2
import math
import threading

pc_kamera = cv2.VideoCapture(0)
pc_kamera.set(3, 640)
pc_kamera.set(4, 480)

# Telefon kamerasını başlatma ve ıp ye göre bağlanma adımı 

# telefon_kamera_adresi = "http://172.16.3.109:8080/video"
logitech_camera = cv2.VideoCapture(1,cv2.CAP_DSHOW)

# hazır eğitilmiş yolo modelim
model = YOLO("yolo-Weights/yolov8n.pt")

sinif_isimleri = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
                  "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                  "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
                  "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                  "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
                  "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                  "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
                  "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                  "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                  "teddy bear", "hair drier", "toothbrush"]

# Initialize variables for fps calculation
start_time_pc = datetime.now()
start_time_telefon = datetime.now()
fps_pc = 0
fps_telefon = 0

while True:
    # laptopumdan görüntü alma
    pc_basari, pc_resim = pc_kamera.read()
    pc_sonuclar = model(pc_resim, stream=True)

    for r in pc_sonuclar:
        pc_kutular = r.boxes

        for pc_kutu in pc_kutular:
            x1, y1, x2, y2 = pc_kutu.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cv2.rectangle(pc_resim, (x1, y1), (x2, y2), (255, 0, 255), 3)

            guven = math.ceil((pc_kutu.conf[0] * 100)) / 100
            print("Güven --->", guven)

            sinif = int(pc_kutu.cls[0])
            print("Sınıf adı -->", sinif_isimleri[sinif])

            metin = f"{sinif_isimleri[sinif]}: {guven:.2f}"
            org = (x1, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontOlcek = 0.5
            renk = (255, 0, 0)
            kalinlik = 1

            cv2.putText(pc_resim, metin, org, font, fontOlcek, renk, kalinlik)

    telefon_basari, telefon_resim = logitech_camera.read()

    if telefon_basari:
        # Görüntü boyutlarını eşitleme kısımı
        telefon_resim = cv2.resize(telefon_resim, (pc_resim.shape[1], pc_resim.shape[0]))

        # Görüntüleri birleştirme işlemi
        birlesik_resim = cv2.hconcat([pc_resim, telefon_resim])

        telefon_sonuclar = model(telefon_resim, stream=True)

        for r in telefon_sonuclar:
            telefon_kutular = r.boxes

            for telefon_kutu in telefon_kutular:
                x1, y1, x2, y2 = telefon_kutu.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                cv2.rectangle(birlesik_resim, (x1 + pc_resim.shape[1], y1), (x2 + pc_resim.shape[1], y2), (255, 0, 255), 3)

                guven = math.ceil((telefon_kutu.conf[0] * 100)) / 100
                print("Güven (Telefon) --->", guven)

                sinif = int(telefon_kutu.cls[0])
                print("Sınıf adı (Telefon) -->", sinif_isimleri[sinif])

                metin = f"{sinif_isimleri[sinif]}: {guven:.2f}"
                org = (x1 + pc_resim.shape[1], y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontOlcek = 0.5
                renk = (255, 0, 0)
                kalinlik = 1

                cv2.putText(birlesik_resim, metin, org, font, fontOlcek, renk, kalinlik)

        # Calculate fps for PC camera
        fps_pc = 1 / (datetime.now() - start_time_pc).total_seconds()
        start_time_pc = datetime.now()

        # Display fps on the PC camera window
        cv2.putText(birlesik_resim, f"FPS (PC): {fps_pc:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Calculate fps for Telefon camera
        fps_telefon = 1 / (datetime.now() - start_time_telefon).total_seconds()
        start_time_telefon = datetime.now()

        # Display fps on the Telefon camera window
        cv2.putText(birlesik_resim, f"FPS (Logitech_camera): {fps_telefon:.2f}", (10 + pc_resim.shape[1], 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # Display the merged image
        cv2.imshow('Bilgisayar ve Telefon Kamerası', birlesik_resim)
    else:
        print("Telefon kamerasından görüntü alınamadı.")

    if cv2.waitKey(1) == ord('q'):
        break


pc_kamera.release()
logitech_camera.release()
cv2.destroyAllWindows()

DETECTION WITH PC AND PHONE CAMERA

In [ ]:
from ultralytics import YOLO
import cv2
import math

# PC Webcam'ini başlat
pc_kamera = cv2.VideoCapture(0)
pc_kamera.set(3, 640)
pc_kamera.set(4, 480)

# Telefon kamerasını başlat
telefon_kamera_adresi = "http://172.16.3.109:8080/video"  # Telefonunuzun IP Webcam uygulamasının sağladığı adresi kullanın
telefon_kamera = cv2.VideoCapture(telefon_kamera_adresi)

# YOLO modelini başlat
model = YOLO("yolo-Weights/yolov8n.pt")

# Nesne sınıfları
sinif_isimleri = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
                  "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                  "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
                  "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                  "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
                  "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                  "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
                  "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                  "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                  "teddy bear", "hair drier", "toothbrush"]  # Sınıf isimleriniz

while True:
    # PC Webcam'inden görüntü al
    pc_basari, pc_resim = pc_kamera.read()
    pc_sonuclar = model(pc_resim, stream=True)

    for r in pc_sonuclar:
        pc_kutular = r.boxes

        for pc_kutu in pc_kutular:
            x1, y1, x2, y2 = pc_kutu.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Algılanan nesnenin sınırlayıcı kutusunu çiz
            cv2.rectangle(pc_resim, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Algılama güvenini hesapla
            guven = math.ceil((pc_kutu.conf[0] * 100)) / 100
            print("guven --->", guven)

            # Algılanan sınıfın adını al
            sinif = int(pc_kutu.cls[0])
            print("Sınıf adı -->", sinif_isimleri[sinif])

            # Algılanan nesnenin sınıf adını ve güvenini ekrana yazdır
            metin = f"{sinif_isimleri[sinif]}: {guven:.2f}"
            org = (x1, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontOlcek = 0.5
            renk = (255, 0, 0)
            kalinlik = 1

            cv2.putText(pc_resim, metin, org, font, fontOlcek, renk, kalinlik)

    # Telefon kamerasından görüntü al
    telefon_basari, telefon_resim = telefon_kamera.read()

    # Telefon kamerasından başarıyla görüntü alındı mı kontrol et
    if telefon_basari:
        # Görüntü boyutlarını eşitle
        telefon_resim = cv2.resize(telefon_resim, (pc_resim.shape[1], pc_resim.shape[0]))

        # Görüntüleri yatay olarak birleştir
        birlesik_resim = cv2.hconcat([pc_resim, telefon_resim])

        # Telefon kamerasındaki görüntü üzerinde de nesne tespiti yap
        telefon_sonuclar = model(telefon_resim, stream=True)

        for r in telefon_sonuclar:
            telefon_kutular = r.boxes

            for telefon_kutu in telefon_kutular:
                x1, y1, x2, y2 = telefon_kutu.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Algılanan nesnenin sınırlayıcı kutusunu çiz
                cv2.rectangle(birlesik_resim, (x1 + pc_resim.shape[1], y1), (x2 + pc_resim.shape[1], y2), (255, 0, 255), 3)

                # Algılama güvenini hesapla
                guven = math.ceil((telefon_kutu.conf[0] * 100)) / 100
                print("Güven (Telefon) --->", guven)

                # Algılanan sınıfın adını al
                sinif = int(telefon_kutu.cls[0])
                print("Sınıf adı (Telefon) -->", sinif_isimleri[sinif])

                # Algılanan nesnenin sınıf adını ve güvenini ekrana yazdır
                metin = f"{sinif_isimleri[sinif]}: {guven:.2f}"
                org = (x1 + pc_resim.shape[1], y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontOlcek = 0.5
                renk = (255, 0, 0)
                kalinlik = 1

                cv2.putText(birlesik_resim, metin, org, font, fontOlcek, renk, kalinlik)

        # Görüntüyü göster
        cv2.imshow('Bilgisayar ve Telefon Kamerası', birlesik_resim)
    else:
        print("Telefon kamerasından görüntü alınamadı.")

    # 'q' tuşuna basıldığında döngüden çık
    if cv2.waitKey(1) == ord('q'):
        break

# Kameraları serbest bırak ve OpenCV pencerelerini kapat
pc_kamera.release()
telefon_kamera.release()
cv2.destroyAllWindows()


THREADS WITH PHONE CAMERA AND PC CAMERA


In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from both cameras and perform object detection
def capture_frames(pc_camera, phone_camera, window_name, model, class_names, target_height=480):
    while True:
        success_pc, frame_pc = pc_camera.read()
        success_phone, frame_phone = phone_camera.read()

        if success_pc and success_phone:
            # Resize frames to have a consistent height
            height_pc, width_pc, _ = frame_pc.shape
            height_phone, width_phone, _ = frame_phone.shape
            target_width_pc = int(target_height * (width_pc / height_pc))
            target_width_phone = int(target_height * (width_phone / height_phone))

            frame_pc = cv2.resize(frame_pc, (target_width_pc, target_height))
            frame_phone = cv2.resize(frame_phone, (target_width_phone, target_height))

            # Concatenate frames horizontally
            combined_frame = np.concatenate((frame_pc, frame_phone), axis=1)

            # Perform object detection on the combined frame
            detect_objects(combined_frame, model, class_names, offset_x=0)

            # Display the combined frame
            cv2.imshow(window_name, combined_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# Start the PC Webcam
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the Phone camera
phone_camera_address = "http://172.16.3.109:8080/video"  # Replace with your phone's IP Webcam address
phone_camera = cv2.VideoCapture(phone_camera_address)

# Start the YOLO model
yolo_model = YOLO("yolo-Weights/yolov8n.pt")

# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Create a thread for capturing frames and performing object detection for both cameras
combined_thread = threading.Thread(target=capture_frames, args=(pc_camera, phone_camera, 'Combined Cameras', yolo_model, class_names))

# Start the thread
combined_thread.start()

# Wait for the thread to finish
combined_thread.join()

# Release the cameras and close OpenCV windows
pc_camera.release()
phone_camera.release()
cv2.destroyAllWindows()


THREAD WITH TWO CAMERA(WEBCAM)fps yok tek thread 

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from both cameras and perform object detection
def capture_frames(pc_camera, web_camera, window_name, model, class_names, target_height=480):
    while True:
        success_pc, frame_pc = pc_camera.read()
        success_web, frame_web = web_camera.read()

        if success_pc and success_web:
            # Resize frames to have a consistent height
            height_pc, width_pc, _ = frame_pc.shape
            height_web, width_web, _ = frame_web.shape
            target_width_pc = int(target_height * (width_pc / height_pc))
            target_width_web = int(target_height * (width_web / height_web))

            frame_pc = cv2.resize(frame_pc, (target_width_pc, target_height))
            frame_web = cv2.resize(frame_web, (target_width_web, target_height))

            # Concatenate frames horizontally
            combined_frame = np.concatenate((frame_pc, frame_web), axis=1)

            # Perform object detection on the combined frame
            detect_objects(combined_frame, model, class_names, offset_x=0)

            # Display the combined frame
            cv2.imshow(window_name, combined_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# Start the PC Webcam
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the web camera
web_camera = cv2.VideoCapture(1,cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# Start the YOLO model
yolo_model = YOLO("yolo-Weights/yolov8n.pt")

# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Create a thread for capturing frames and performing object detection for both cameras
combined_thread = threading.Thread(target=capture_frames, args=(pc_camera, web_camera, 'Combined Cameras', yolo_model, class_names))

# Start the thread
combined_thread.start()

# Wait for the thread to finish
combined_thread.join()

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
cv2.destroyAllWindows()


WITH FPS

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from both cameras and perform object detection
def capture_frames(pc_camera, web_camera, window_name, model, class_names, target_height=480):
    start_time_pc = time.time()
    frame_count_pc = 0

    start_time_web = time.time()
    frame_count_web = 0

    while True:
        success_pc, frame_pc = pc_camera.read()
        success_web, frame_web = web_camera.read()

        if success_pc and success_web:
            # Resize frames to have a consistent height
            height_pc, width_pc, _ = frame_pc.shape
            height_web, width_web, _ = frame_web.shape
            target_width_pc = int(target_height * (width_pc / height_pc))
            target_width_web = int(target_height * (width_web / height_web))

            frame_pc = cv2.resize(frame_pc, (target_width_pc, target_height))
            frame_web = cv2.resize(frame_web, (target_width_web, target_height))

            # Concatenate frames horizontally
            combined_frame = np.hstack((frame_pc, frame_web))

            # Perform object detection on the combined frame
            detect_objects(combined_frame, model, class_names, offset_x=0)

            # Calculate and display FPS for PC Camera
            frame_count_pc += 1
            elapsed_time_pc = time.time() - start_time_pc
            fps_pc = frame_count_pc / elapsed_time_pc
            cv2.putText(combined_frame, f"FPS PC: {fps_pc:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Calculate and display FPS for Web Camera
            frame_count_web += 1
            elapsed_time_web = time.time() - start_time_web
            fps_web = frame_count_web / elapsed_time_web
            cv2.putText(combined_frame, f"FPS Web: {fps_web:.2f}", (target_width_pc + 10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the combined frame in the window
            cv2.imshow(window_name, combined_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# Start the PC Webcam
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the Web camera
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# Start the YOLO model
yolo_model = YOLO("yolo-Weights/yolov8n.pt")

# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Create a thread for capturing frames and performing object detection for both cameras
combined_thread = threading.Thread(target=capture_frames, args=(pc_camera, web_camera, 'Combined Cameras', yolo_model, class_names))

# Start the thread
combined_thread.start()
# Wait for the thread to finish
combined_thread.join()

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
cv2.destroyAllWindows()


sadeleştirilmiş

In [ ]:
import cv2
import math
import threading
import numpy as np
import time
from ultralytics import YOLO

class ObjectDetector:
    def __init__(self, model_path, class_names):
        self.model = YOLO(model_path)
        self.class_names = class_names

    def detect_objects(self, frame, offset_x=0):
        results = self.model(frame, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Draw bounding box for the detected object
                cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

                # Calculate detection confidence
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Get the class name of the detected object
                class_id = int(box.cls[0])

                # Display the class name and confidence of the detected object on the frame
                text = f"{self.class_names[class_id]}: {confidence:.2f}"
                org = (x1 + offset_x, y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                color = (255, 0, 0)
                thickness = 1

                cv2.putText(frame, text, org, font, font_scale, color, thickness)

class Camera:
    def __init__(self, camera_id, target_height=480):
        self.camera = cv2.VideoCapture(camera_id,cv2.CAP_DSHOW)
        self.camera.set(3, 640)
        self.camera.set(4, target_height)

    def read_frame(self):
        success, frame = self.camera.read()
        return success, frame

    def release(self):
        self.camera.release()

class MultiCameraObjectDetection:
    def __init__(self, pc_camera_id, web_camera_id, detector, window_name):
        self.pc_camera = Camera(pc_camera_id)
        self.web_camera = Camera(web_camera_id)
        self.detector = detector
        self.window_name = window_name

    def capture_frames(self):
        start_time_pc = time.time()
        frame_count_pc = 0

        start_time_web = time.time()
        frame_count_web = 0

        while True:
            success_pc, frame_pc = self.pc_camera.read_frame()
            success_web, frame_web = self.web_camera.read_frame()

            if success_pc and success_web:
                # Resize frames to have a consistent height
                height_pc, width_pc, _ = frame_pc.shape
                height_web, width_web, _ = frame_web.shape
                target_width_pc = int(height_pc * (width_pc / height_pc))
                target_width_web = int(height_web * (width_web / height_web))

                frame_pc = cv2.resize(frame_pc, (target_width_pc, height_pc))
                frame_web = cv2.resize(frame_web, (target_width_web, height_web))

                # Concatenate frames horizontally
                combined_frame = np.hstack((frame_pc, frame_web))

                # Perform object detection on the combined frame
                self.detector.detect_objects(combined_frame, offset_x=0)

                # Calculate and display FPS for PC Camera
                frame_count_pc += 1
                elapsed_time_pc = time.time() - start_time_pc
                fps_pc = frame_count_pc / elapsed_time_pc
                cv2.putText(combined_frame, f"FPS PC: {fps_pc:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Calculate and display FPS for Web Camera
                frame_count_web += 1
                elapsed_time_web = time.time() - start_time_web
                fps_web = frame_count_web / elapsed_time_web
                cv2.putText(combined_frame, f"FPS Web: {fps_web:.2f}", (target_width_pc + 10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Display the combined frame in the window
                cv2.imshow(self.window_name, combined_frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        self.pc_camera.release()
        self.web_camera.release()
        cv2.destroyAllWindows()

# Start the YOLO model
yolo_model_path = "yolo-Weights/yolov8n.pt"
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

detector = ObjectDetector(yolo_model_path, class_names)

# Create a thread for capturing frames and performing object detection for both cameras
multi_cam_detector = MultiCameraObjectDetection(0, 1, detector, 'Combined Cameras')
combined_thread = threading.Thread(target=multi_cam_detector.capture_frames)

# Start the thread
combined_thread.start()
# Wait for the thread to finish
combined_thread.join()


 iki thread ayrı ekranlar

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(camera, window_name, model, class_names, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        success, frame = camera.read()

        if success:
            # Resize frame to have a consistent height
            height, width, _ = frame.shape
            target_width = int(target_height * (width / height))
            #target_width = 480
            frame = cv2.resize(frame, (target_width, target_height))

            # Perform object detection on the frame
            detect_objects(frame, model, class_names, offset_x=0)

            # Calculate and display FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the frame in the window
            cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# webcamı başlatma işlemi
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the Web camera
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# Start the YOLO model
yolo_model = YOLO("yolo-Weights/yolov8n.pt")
yolo_model_2 = YOLO("yolo-Weights/yolov8n2.pt")
    
# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Ayrı ayrı pencereler için isimler
window_name_pc = 'PC Camera'
window_name_web = 'Web Camera'

# Create threads for capturing frames and performing object detection for both cameras
pc_thread = threading.Thread(target=capture_frames, args=(pc_camera, window_name_pc, yolo_model, class_names))
web_thread = threading.Thread(target=capture_frames, args=(web_camera, window_name_web, yolo_model_2, class_names))

# Start the threads
pc_thread.start()
web_thread.start()

# Wait for the threads to finish
pc_thread.join()
web_thread.join()

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
cv2.destroyAllWindows()

iki thread ayrı ekran fps iyi

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(camera, window_name, model, class_names, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        success, frame = camera.read()

        if success:
            # Resize frame to have a consistent height
            height, width, _ = frame.shape
            target_width = int(target_height * (width / height))
            frame = cv2.resize(frame, (target_width, target_height))

            # Perform object detection on the frame
            detect_objects(frame, model, class_names, offset_x=0)

            # Calculate and display FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the frame in the window
            cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# Start the PC Webcam
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the Web camera
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# Start the YOLO models
yolo_model_pc = YOLO("yolo-Weights/yolov8n.pt")
yolo_model_web = YOLO("yolo-Weights/yolov8n2.pt")

# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Create a thread for capturing frames and performing object detection for both cameras
pc_thread = threading.Thread(target=capture_frames, args=(pc_camera, 'PC Camera', yolo_model_pc, class_names))
web_thread = threading.Thread(target=capture_frames, args=(web_camera, 'Web Camera', yolo_model_web, class_names))

# Start the threads
pc_thread.start()
web_thread.start()

# Wait for the threads to finish
pc_thread.join()
web_thread.join()

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
cv2.destroyAllWindows()


iki thread aynı ekran

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(camera, model, class_names, offset_x, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        success, frame = camera.read()

        if success:
            # Resize frame to have a consistent height
            height, width, _ = frame.shape
            target_width = int(target_height * (width / height))
            frame = cv2.resize(frame, (target_width, target_height))

            # Perform object detection on the frame
            detect_objects(frame, model, class_names, offset_x)

            # Calculate and display FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10 + offset_x, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            return frame

# Start the PC Webcam
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the Web camera
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# Start the YOLO models
yolo_model = YOLO("yolo-Weights/yolov8n.pt")


# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]


## Function to capture frames and perform object detection
def capture_frames_and_detect(camera, model, class_names, offset_x, frames_buffer, lock, target_height=480):
    while True:
        frame = capture_frames(camera, model, class_names, offset_x, target_height)
        with lock:
            frames_buffer[offset_x] = frame.copy()  # Ensure thread safety with a deep copy
            # Detect objects on the frame
            detect_objects(frames_buffer[offset_x], model, class_names, offset_x)

# Create a lock and frames buffer
lock = threading.Lock()
frames_buffer = {}

pc_thread = threading.Thread(target=capture_frames_and_detect, args=(pc_camera, yolo_model, class_names, 0, frames_buffer, lock))
web_thread = threading.Thread(target=capture_frames_and_detect, args=(web_camera, yolo_model, class_names,640, frames_buffer, lock))

# Start the threads
pc_thread.start()
web_thread.start()

# Display frames
while True:
    with lock:
        if 0 in frames_buffer and 640 in frames_buffer:
            combined_frame = np.hstack((frames_buffer[0], frames_buffer[640]))
            cv2.imshow('Combined Cameras', combined_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
cv2.destroyAllWindows()


In [5]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(camera, model, class_names, offset_x, target_height=480):
    while True:
        start_time = time.time()
        frame_count = 0

        success, frame = camera.read()

        if success:
            # Resize frame to have a consistent height
            height, width, _ = frame.shape
            target_width = int(target_height * (width / height))
            frame = cv2.resize(frame, (target_width, target_height))

            # Perform object detection on the frame
            detect_objects(frame, model, class_names, offset_x)

            # Calculate and display FPS
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10 + offset_x, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            yield frame

# Start the PC Webcam
pc_camera = cv2.VideoCapture(0)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Start the Web camera
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# Start the YOLO models
yolo_model = YOLO("yolo-Weights/yolov8n.pt")

# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]


## Function to capture frames and perform object detection
def capture_frames_and_detect(camera, model, class_names, offset_x, frames_buffer, lock, target_height=480):
    for frame in capture_frames(camera, model, class_names, offset_x, target_height):
        with lock:
            frames_buffer[offset_x] = frame.copy()  # Ensure thread safety with a deep copy
            # Detect objects on the frame
            detect_objects(frames_buffer[offset_x], model, class_names, offset_x)

# Create a lock and frames buffer
lock = threading.Lock()
frames_buffer = {}

pc_thread = threading.Thread(target=capture_frames_and_detect, args=(pc_camera, yolo_model, class_names, 0, frames_buffer, lock))
web_thread = threading.Thread(target=capture_frames_and_detect, args=(web_camera, yolo_model, class_names, 640, frames_buffer, lock))

# Start the threads
pc_thread.start()
web_thread.start()

# Display frames
while True:
    with lock:
        if 0 in frames_buffer and 640 in frames_buffer:
            combined_frame = np.hstack((frames_buffer[0], frames_buffer[640]))
            cv2.imshow('Combined Cameras', combined_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
cv2.destroyAllWindows()




0: 480x640 1 person, 102.6ms
Speed: 4.4ms preprocess, 102.6ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 103.2ms
Speed: 2.0ms preprocess, 103.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

Ultralytics YOLOv8.1.9 🚀 Python-3.9.18 torch-2.2.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
0: 480x640 1 person, 105.0ms
Speed: 2.0ms preprocess, 105.0ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 92.1ms
Speed: 1.0ms preprocess, 92.1ms inference, 4.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 91.1ms
Speed: 1.0ms preprocess, 91.1ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 98.0ms
Speed: 2.0ms preprocess, 98.0ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
0: 480x640 1 person, 95.0ms
Speed: 1.0ms preprocess, 95.0ms i

3 pencere 

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import numpy as np
import time


def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            
            confidence = math.ceil((box.conf[0] * 100)) / 100

       
            class_id = int(box.cls[0])

            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)
            



def capture_frames(pc_camera, web_camera,phone_camera , window_name, model, class_names, target_height=480):
    start_time_pc = time.time()
    frame_count_pc = 0

    start_time_web = time.time()
    frame_count_web = 0
    
    start_time_phone = time.time()
    frame_count_phone = 0

    while True:
        success_pc, frame_pc = pc_camera.read()
        success_web, frame_web = web_camera.read()
        success_phone_camera, frame_phone = phone_camera.read()

        if success_pc and success_web and success_phone_camera:
            
            height_pc, width_pc, _ = frame_pc.shape
            height_web, width_web, _ = frame_web.shape
            height_phone,with_phone,_ = frame_phone.shape
            
            
            target_width_pc = int(target_height * (width_pc / height_pc))
            target_width_web = int(target_height * (width_web / height_web))
            target_width_phone= int(target_height * (with_phone / height_phone))
            
            frame_pc = cv2.resize(frame_pc, (target_width_pc, target_height))
            frame_web = cv2.resize(frame_web, (target_width_web, target_height))
            frame_phone= cv2.resize(frame_phone, (target_width_phone, target_height))

        
            combined_frame = np.hstack((frame_pc, frame_web,frame_phone))

            detect_objects(combined_frame, model, class_names, offset_x=0)

          
            frame_count_pc += 1
            elapsed_time_pc = time.time() - start_time_pc
            fps_pc = frame_count_pc / elapsed_time_pc
            cv2.putText(combined_frame, f"FPS PC: {fps_pc:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            frame_count_web += 1
            elapsed_time_web = time.time() - start_time_web
            fps_web = frame_count_web / elapsed_time_web
            cv2.putText(combined_frame, f"FPS Web: {fps_web:.2f}", (target_width_pc + 10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            
            frame_count_phone += 1
            elapsed_time_phone = time.time() - start_time_phone
            fps_phone = frame_count_phone / elapsed_time_phone
            cv2.putText(combined_frame, f"FPS PHONE: {fps_phone:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            
            combined_frame=cv2.resize(combined_frame, (1920, 1080))
            cv2.imshow(window_name, combined_frame)
            #cv2.resizeWindow(window_name, 800, 600, combined_frame)
            #cv2.imshow(window_name, combined_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

# Laptop kamera başlatma işlemi
pc_camera = cv2.VideoCapture(0,cv2.CAP_DSHOW)
pc_camera.set(3, 640)
pc_camera.set(4, 480)

# Web kamera başlatma işlemi
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
web_camera.set(3, 640)
web_camera.set(4, 480)

# telefon kamerasını başlatmak için
Phone_Camera_Adress = "http://172.16.3.109:8080/video" 
phone_camera= cv2.VideoCapture(Phone_Camera_Adress) #Bekleme süresini azaltmak için kullanılır.
phone_camera.set(3, 640)
phone_camera.set(4, 480)

#Hazır eğitilmiş Yolo modelimi Başlatma işlemi.
yolo_model = YOLO("yolo-Weights/yolov8n.pt")
    

class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]


combined_thread = threading.Thread(target=capture_frames, args=(pc_camera, web_camera,phone_camera ,'Combined Cameras', yolo_model, class_names))


combined_thread.start()


combined_thread.join()

pc_camera.release()
web_camera.release()
phone_camera.release()
cv2.destroyAllWindows()

3 pencere ayrı ayrı threadli

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(camera, window_name, model, class_names, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        
        success, frame = camera.read()

        if success:
            # Resize frame to have a consistent height
            height, width, _ = frame.shape
            target_width = int(target_height * (width / height))
            frame = cv2.resize(frame, (target_width, target_height))

            # Perform object detection on the frame
            detect_objects(frame, model, class_names, offset_x=0)

            #fps
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            

            # Display the frame in the window
            cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    camera.release()

# webcamı başlatma işlemi
pc_camera = cv2.VideoCapture(0)

# Start the Web camera
web_camera = cv2.VideoCapture(1,cv2.CAP_DSHOW)

# Start phone camera
phone_camera_address = "http://172.16.3.109:8080/video"
phone_camera = cv2.VideoCapture(phone_camera_address)

# Start the YOLO models
yolo_model_pc = YOLO("yolo-Weights/yolov8n.pt")
yolo_model_web = YOLO("yolo-Weights/yolov8n.pt")
yolo_model_phone = YOLO("yolo-Weights/yolov8n.pt")

# Object classes
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Ayrı ayrı pencereler için isimler
window_name_pc = 'PC Camera'
window_name_web = 'Web Camera'
window_name_phone = 'Phone Camera'

# Create threads for capturing frames and performing object detection for each camera
pc_thread = threading.Thread(target=capture_frames, args=(pc_camera, window_name_pc, yolo_model_pc, class_names))
web_thread = threading.Thread(target=capture_frames, args=(web_camera, window_name_web, yolo_model_web, class_names))
phone_thread = threading.Thread(target=capture_frames, args=(phone_camera, window_name_phone, yolo_model_phone, class_names))

# Start the threads
pc_thread.start()
web_thread.start()
phone_thread.start()

# Wait for the threads to finish
pc_thread.join()
web_thread.join()
phone_thread.join()

# Release the cameras and close OpenCV windows
pc_camera.release()
web_camera.release()
phone_camera.release()
cv2.destroyAllWindows()


3 pencere tek thread

In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(cameras, window_names, model, class_names, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        frames = []
        for camera in cameras:
            success, frame = camera.read()
            if success:
                # Resize frame to have a consistent height
                height, width, _ = frame.shape
                target_width = int(target_height * (width / height))
                frame = cv2.resize(frame, (target_width, target_height))
                frames.append(frame)

        # Perform object detection on each frame
        for frame, window_name in zip(frames, window_names):
            detect_objects(frame, model, class_names)
            cv2.imshow(window_name, frame)

        # fps
        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time
        print(f"FPS: {fps:.2f}")

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    for camera in cameras:
        camera.release()
    cv2.destroyAllWindows()

# Start the YOLO models
yolo_model = YOLO("yolo-Weights/yolov8n.pt")

class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Start the camera threads
def start_camera_thread(cameras, window_names):
    capture_thread = threading.Thread(target=capture_frames, args=(cameras, window_names, yolo_model, class_names))
    capture_thread.start()
    return capture_thread

# Open cameras
pc_camera = cv2.VideoCapture(0)
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
phone_camera_address = "http://172.16.3.109:8080/video"
phone_camera = cv2.VideoCapture(phone_camera_address)

# Window names
window_names = ['PC Camera', 'Web Camera', 'Phone Camera']

# Start the camera thread
camera_thread = start_camera_thread([pc_camera, web_camera, phone_camera], window_names)

# Wait for the thread to finish
camera_thread.join()


In [ ]:
from ultralytics import YOLO
import cv2
import math
import threading
import time

# Function: Perform object detection on a specific frame
def detect_objects(frame, model, class_names, offset_x=0):
    results = model(frame, stream=True)

    for r in results:
        boxes = r.boxes

        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            # Draw bounding box for the detected object
            cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

            # Calculate detection confidence
            confidence = math.ceil((box.conf[0] * 100)) / 100

            # Get the class name of the detected object
            class_id = int(box.cls[0])

            # Display the class name and confidence of the detected object on the frame
            text = f"{class_names[class_id]}: {confidence:.2f}"
            org = (x1 + offset_x, y1 - 10)
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            color = (255, 0, 0)
            thickness = 1

            cv2.putText(frame, text, org, font, font_scale, color, thickness)

# Function: Read frames from camera and perform object detection
def capture_frames(cameras, window_names, model, class_names, target_height=480):
    start_time = time.time()
    frame_count = 0

    while True:
        frames = []
        for camera in cameras:
            success, frame = camera.read()
            if success:
                # Resize frame to have a consistent height
                height, width, _ = frame.shape
                target_width = int(target_height * (width / height))
                frame = cv2.resize(frame, (target_width, target_height))
                frames.append(frame)

        # Perform object detection on each frame
        for frame, window_name in zip(frames, window_names):
            detect_objects(frame, model, class_names)
            # fps
            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    for camera in cameras:
        camera.release()
    cv2.destroyAllWindows()

# Start the YOLO models
yolo_model = YOLO("yolo-Weights/yolov8n.pt")

class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

# Start the camera threads
def start_camera_thread(cameras, window_names):
    capture_thread = threading.Thread(target=capture_frames, args=(cameras, window_names, yolo_model, class_names))
    capture_thread.start()
    return capture_thread

# Open cameras
pc_camera = cv2.VideoCapture(0)
web_camera = cv2.VideoCapture(1, cv2.CAP_DSHOW)
phone_camera_address = "http://172.16.3.109:8080/video"
phone_camera = cv2.VideoCapture(phone_camera_address)

# Window names
window_names = ['PC Camera', 'Web Camera', 'Phone Camera']

# Start the camera thread
camera_thread = start_camera_thread([pc_camera, web_camera, phone_camera], window_names)

# Wait for the thread to finish
camera_thread.join()


herbir kamera için thread(pc webcam)

In [ ]:
import cv2
import threading
import time
from ultralytics import YOLO

class ObjectDetector:
    def __init__(self, model_path, class_names):
        self.model = YOLO(model_path)
        self.class_names = class_names

    def detect_objects(self, frame, offset_x=0):
        results = self.model(frame, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Draw bounding box for the detected object
                cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

                # Calculate detection confidence
                confidence = float(box.conf[0])

                # Get the class name of the detected object
                class_id = int(box.cls[0])

                # Display the class name and confidence of the detected object on the frame
                text = f"{self.class_names[class_id]}: {confidence:.2f}"
                org = (x1 + offset_x, y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                color = (255, 0, 0)
                thickness = 1

                cv2.putText(frame, text, org, font, font_scale, color, thickness)

class Camera:
    def __init__(self, camera_id, target_height=480):
        self.camera = cv2.VideoCapture(camera_id, cv2.CAP_DSHOW)
        if not self.camera.isOpened():
            raise ValueError(f"Could not open camera with ID {camera_id}")
        self.camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.camera.set(cv2.CAP_PROP_FRAME_HEIGHT, target_height)

    def read_frame(self):
        success, frame = self.camera.read()
        return success, frame

    def release(self):
        self.camera.release()

class MultiCameraObjectDetection:
    def __init__(self, pc_camera_id, web_camera_id, detector, window_name):
        self.pc_camera = Camera(pc_camera_id)
        self.web_camera = Camera(web_camera_id)
        self.detector = detector
        self.window_name = window_name

    def capture_pc_frames(self):
        start_time_pc = time.time()
        frame_count_pc = 0

        while True:
            success_pc, frame_pc = self.pc_camera.read_frame()

            if success_pc:
                frame_pc = cv2.resize(frame_pc, (640, 480))  # Resize the frame

                self.detector.detect_objects(frame_pc, offset_x=0)

                frame_count_pc += 1
                elapsed_time_pc = time.time() - start_time_pc
                fps_pc = frame_count_pc / elapsed_time_pc
                cv2.putText(frame_pc, f"FPS PC: {fps_pc:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                cv2.imshow(self.window_name + " PC", frame_pc)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        self.pc_camera.release()

    def capture_web_frames(self):
        start_time_web = time.time()
        frame_count_web = 0

        while True:
            success_web, frame_web = self.web_camera.read_frame()

            if success_web:
                frame_web = cv2.resize(frame_web, (640, 480))  # Resize the frame

                self.detector.detect_objects(frame_web, offset_x=0)

                frame_count_web += 1
                elapsed_time_web = time.time() - start_time_web
                fps_web = frame_count_web / elapsed_time_web
                cv2.putText(frame_web, f"FPS Web: {fps_web:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                cv2.imshow(self.window_name + " Web", frame_web)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        self.web_camera.release()

# Start the YOLO model
yolo_model_path = "yolo-Weights/yolov8n.pt"
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

detector = ObjectDetector(yolo_model_path, class_names)

# Create threads for capturing frames and performing object detection for both cameras
multi_cam_detector = MultiCameraObjectDetection(0, 1, detector, ' Camera')
pc_thread = threading.Thread(target=multi_cam_detector.capture_pc_frames)
web_thread = threading.Thread(target=multi_cam_detector.capture_web_frames)

# Start the threads
pc_thread.start()
web_thread.start()

# Wait for the threads to finish
pc_thread.join()
web_thread.join()


pencereler sırasıyla açılıyor fps yüksek


In [ ]:
import cv2
import math
import threading
import numpy as np
import time
from ultralytics import YOLO

class ObjectDetector:
    def __init__(self, model_path, class_names):
        self.model = YOLO(model_path)
        self.class_names = class_names

    def detect_objects(self, frame, offset_x=0):
        results = self.model(frame, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Draw bounding box for the detected object
                cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

                # Calculate detection confidence
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Get the class name of the detected object
                class_id = int(box.cls[0])

                # Display the class name and confidence of the detected object on the frame
                text = f"{self.class_names[class_id]}: {confidence:.2f}"
                org = (x1 + offset_x, y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                color = (255, 0, 0)
                thickness = 1

                cv2.putText(frame, text, org, font, font_scale, color, thickness)

class Camera:
    def __init__(self, camera_id, target_height=480):
        self.camera = cv2.VideoCapture(camera_id,cv2.CAP_DSHOW)
        if not self.camera.isOpened():
            raise ValueError(f"Could not open camera with ID {camera_id}")
        self.camera.set(3, 640)
        self.camera.set(4, target_height)

    def read_frame(self):
        success, frame = self.camera.read()
        return success, frame

    def release(self):
        self.camera.release()

class MultiCameraObjectDetection:
    def __init__(self, pc_camera_id, web_camera_id, detector, window_name):
        self.pc_camera = Camera(pc_camera_id)
        self.web_camera = Camera(web_camera_id)
        self.detector = detector
        self.window_name = window_name

    def capture_pc_frames(self):
        start_time_pc = time.time()
        frame_count_pc = 0

        while True:
            success_pc, frame_pc = self.pc_camera.read_frame()

            if success_pc:
                height_pc, width_pc, _ = frame_pc.shape
                target_width_pc = int(height_pc * (width_pc / height_pc))
                frame_pc = cv2.resize(frame_pc, (target_width_pc, height_pc))

                self.detector.detect_objects(frame_pc, offset_x=0)

                frame_count_pc += 1
                elapsed_time_pc = time.time() - start_time_pc
                fps_pc = frame_count_pc / elapsed_time_pc
                cv2.putText(frame_pc, f"FPS PC: {fps_pc:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                cv2.imshow(self.window_name + " PC", frame_pc)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        self.pc_camera.release()

    def capture_web_frames(self):
        start_time_web = time.time()
        frame_count_web = 0

        while True:
            success_web, frame_web = self.web_camera.read_frame()

            if success_web:
                height_web, width_web, _ = frame_web.shape
                target_width_web = int(height_web * (width_web / height_web))
                frame_web = cv2.resize(frame_web, (target_width_web, height_web))

                self.detector.detect_objects(frame_web, offset_x=0)

                frame_count_web += 1
                elapsed_time_web = time.time() - start_time_web
                fps_web = frame_count_web / elapsed_time_web
                cv2.putText(frame_web, f"FPS Web: {fps_web:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                cv2.imshow(self.window_name + " Web", frame_web)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        self.web_camera.release()

# Start the YOLO model
yolo_model_path = "yolo-Weights/yolov8n.pt"
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

detector = ObjectDetector(yolo_model_path, class_names)

# Create multi-camera object detection instance
multi_cam_detector = MultiCameraObjectDetection(0, 1, detector, 'Combined Cameras')

# Start the threads for capturing frames and performing object detection
pc_thread = threading.Thread(target=MultiCameraObjectDetection(0, 1, detector, 'Combined Cameras').capture_pc_frames)
web_thread = threading.Thread(target=MultiCameraObjectDetection(0, 1, detector, 'Combined Cameras').capture_web_frames)

pc_thread.start()
web_thread.start()

pc_thread.join()
web_thread.join()

#multi_cam_detector.capture_pc_frames()
#multi_cam_detector.capture_web_frames()

# Release resources
cv2.destroyAllWindows()


**************ANOTHER WAY********* 2 thread ayrı ekran

In [ ]:
import cv2
import math
import concurrent.futures
import numpy as np
import time
from ultralytics import YOLO

class ObjectDetector:
    def __init__(self, model_path, class_names):
        self.model = YOLO(model_path)
        self.class_names = class_names

    def detect_objects(self, frame, offset_x=0):
        results = self.model(frame, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Draw bounding box for the detected object
                cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

                # Calculate detection confidence
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Get the class name of the detected object
                class_id = int(box.cls[0])

                # Display the class name and confidence of the detected object on the frame
                text = f"{self.class_names[class_id]}: {confidence:.2f}"
                org = (x1 + offset_x, y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                color = (255, 0, 0)
                thickness = 1

                cv2.putText(frame, text, org, font, font_scale, color, thickness)

class Camera:
    def __init__(self, camera_id, target_height=480):
        self.camera = cv2.VideoCapture(camera_id)
        if not self.camera.isOpened():
            raise ValueError(f"Could not open camera with ID {camera_id}")
        self.camera.set(3, 640)
        self.camera.set(4, target_height)

    def read_frame(self):
        success, frame = self.camera.read()
        return success, frame

    def release(self):
        self.camera.release()

def capture_frames(camera, detector, offset_x, window_name):
    start_time = time.time()
    frame_count = 0

    while True:
        success, frame = camera.read_frame()

        if success:
            height, width, _ = frame.shape
            target_width = int(height * (width / height))
            frame = cv2.resize(frame, (target_width, height))

            detector.detect_objects(frame, offset_x=offset_x)

            frame_count += 1
            elapsed_time = time.time() - start_time
            fps = frame_count / elapsed_time
            cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    camera.release()

# Start the YOLO model
yolo_model_path = "yolo-Weights/yolov8n.pt"
yolo_model_path2 = "yolo-Weights/yolov8n2.pt"

class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

detector = ObjectDetector(yolo_model_path, class_names)
detector1=ObjectDetector(yolo_model_path2, class_names)

# Create camera objects
pc_camera = Camera(0)
web_camera = Camera(1,cv2.CAP_DSHOW)

# Create thread pool executor
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    executor.submit(capture_frames, pc_camera, detector, offset_x=0, window_name='PC Camera')
    executor.submit(capture_frames, web_camera, detector1, offset_x=0, window_name='Web Camera')

# Wait for the threads to finish
cv2.destroyAllWindows()


diğer yol tek thread iki ekran

In [ ]:
import cv2
import math
import concurrent.futures
import numpy as np
import time
from ultralytics import YOLO

class ObjectDetector:
    def __init__(self, model_path, class_names):
        self.model = YOLO(model_path)
        self.class_names = class_names

    def detect_objects(self, frame, offset_x=0):
        results = self.model(frame, stream=True)

        for r in results:
            boxes = r.boxes

            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Draw bounding box for the detected object
                cv2.rectangle(frame, (x1 + offset_x, y1), (x2 + offset_x, y2), (255, 0, 255), 3)

                # Calculate detection confidence
                confidence = math.ceil((box.conf[0] * 100)) / 100

                # Get the class name of the detected object
                class_id = int(box.cls[0])

                # Display the class name and confidence of the detected object on the frame
                text = f"{self.class_names[class_id]}: {confidence:.2f}"
                org = (x1 + offset_x, y1 - 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                color = (255, 0, 0)
                thickness = 1

                cv2.putText(frame, text, org, font, font_scale, color, thickness)

class Camera:
    def __init__(self, camera_id, target_height=480):
        self.camera = cv2.VideoCapture(camera_id)
        if not self.camera.isOpened():
            raise ValueError(f"Could not open camera with ID {camera_id}")
        self.camera.set(3, 640)
        self.camera.set(4, target_height)

    def read_frame(self):
        success, frame = self.camera.read()
        return success, frame

    def release(self):
        self.camera.release()

def capture_frames(cameras, detector, window_names):
    start_time = time.time()
    frame_count = 0

    while True:
        frames = [camera.read_frame() for camera in cameras]
        successes, frames = zip(*frames)

        if all(successes):
            frames_resized = [cv2.resize(frame, (640, 480)) for frame in frames]

            for frame, window_name in zip(frames_resized, window_names):
                detector.detect_objects(frame)
                frame_count += 1
                elapsed_time = time.time() - start_time
                fps = frame_count / elapsed_time
                cv2.putText(frame, f"FPS: {fps:.2f}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.imshow(window_name, frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    for camera in cameras:
        camera.release()

# Start the YOLO model
yolo_model_path = "yolo-Weights/yolov8n.pt"
class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
               "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
               "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
               "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
               "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
               "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
               "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
               "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
               "teddy bear", "hair drier", "toothbrush"]

detector = ObjectDetector(yolo_model_path, class_names)

# Create camera objects
pc_camera = Camera(0)
web_camera = Camera(1,cv2.CAP_DSHOW)

# Ayrı ayrı pencereler için isimler
window_names = ['PC Camera', 'Web Camera']

# Create thread pool executor
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    executor.submit(capture_frames, [pc_camera, web_camera], detector, window_names)

# Wait for the thread to finish
cv2.destroyAllWindows()
